In [35]:
import requests
from bs4 import BeautifulSoup

url = "https://www.nytimes.com/" 
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
try:
    response = requests.get(url, headers=headers, timeout=10)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'lxml')

    headings = soup.find_all(['h1', 'h2'])
    title_candidates = [(h.text.strip(), len(h.text.strip())) for h in headings if h.text.strip()]
    longest_title = max(title_candidates, key=lambda x: x[1])[0] if title_candidates else "No title found"
    print("Extracted Title:", longest_title)

    paragraphs = soup.find_all('p')
    body_candidates = [p.text.strip() for p in paragraphs if len(p.text.strip()) > 100]
    article_body = ' '.join(body_candidates[:5]) if body_candidates else "No body found"  # Limit to 5 paragraphs
    print("Extracted Body Snippet:", article_body[:200] + "..." if article_body else article_body)

except requests.RequestException as e:
    print("Error fetching URL:", e)

Extracted Title: AudioPodcasts and narrated articles
Extracted Body Snippet: New recipes, easy dinner ideas and smart kitchen tips from Melissa Clark, Sam Sifton and our New York Times Cooking editors. New recipes, easy dinner ideas and smart kitchen tips from Melissa Clark, S...


In [36]:
import requests
from bs4 import BeautifulSoup

url = "https://www.theguardian.com/" 
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
try:
    response = requests.get(url, headers=headers, timeout=10)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'lxml')

    headings = soup.find_all(['h1', 'h2'])
    title_candidates = [(h.text.strip(), len(h.text.strip())) for h in headings if h.text.strip()]
    longest_title = max(title_candidates, key=lambda x: x[1])[0] if title_candidates else "No title found"
    print("Extracted Title:", longest_title)

    paragraphs = soup.find_all('p')
    body_candidates = [p.text.strip() for p in paragraphs if len(p.text.strip()) > 100]
    article_body = ' '.join(body_candidates[:5]) if body_candidates else "No body found"  # Limit to 5 paragraphs
    print("Extracted Body Snippet:", article_body[:200] + "..." if article_body else article_body)

except requests.RequestException as e:
    print("Error fetching URL:", e)

Extracted Title: More climate crisis & environment
Extracted Body Snippet: No body found...


In [41]:
import requests
from bs4 import BeautifulSoup

url = "https://tass.com/" 
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
try:
    response = requests.get(url, headers=headers, timeout=10)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'lxml')

    headings = soup.find_all(['h1', 'h2'])
    title_candidates = [(h.text.strip(), len(h.text.strip())) for h in headings if h.text.strip()]
    longest_title = max(title_candidates, key=lambda x: x[1])[0] if title_candidates else "No title found"
    print("Extracted Title:", longest_title)

    paragraphs = soup.find_all('p')
    body_candidates = [p.text.strip() for p in paragraphs if len(p.text.strip()) > 100]
    article_body = ' '.join(body_candidates[:5]) if body_candidates else "No body found"  # Limit to 5 paragraphs
    print("Extracted Body Snippet:", article_body[:200] + "..." if article_body else article_body)

except requests.RequestException as e:
    print("Error fetching URL:", e)

Extracted Title: Trump vows to impose sanctions on Russia once NATO countries stop buying its oil
Extracted Body Snippet: No body found...


In [38]:
import requests
from bs4 import BeautifulSoup

url = "https://www.rbc.ru/" 
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
try:
    response = requests.get(url, headers=headers, timeout=10)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'lxml')

    headings = soup.find_all(['h1', 'h2'])
    title_candidates = [(h.text.strip(), len(h.text.strip())) for h in headings if h.text.strip()]
    longest_title = max(title_candidates, key=lambda x: x[1])[0] if title_candidates else "No title found"
    print("Extracted Title:", longest_title)

    paragraphs = soup.find_all('p')
    body_candidates = [p.text.strip() for p in paragraphs if len(p.text.strip()) > 100]
    article_body = ' '.join(body_candidates[:5]) if body_candidates else "No body found"  # Limit to 5 paragraphs
    print("Extracted Body Snippet:", article_body[:200] + "..." if article_body else article_body)

except requests.RequestException as e:
    print("Error fetching URL:", e)

Extracted Title: Новости
Extracted Body Snippet: No body found...


In [40]:
import requests
from bs4 import BeautifulSoup

url = "https://middle-east-online.com/" 
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
try:
    response = requests.get(url, headers=headers, timeout=10)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'lxml')

    headings = soup.find_all(['h1', 'h2'])
    title_candidates = [(h.text.strip(), len(h.text.strip())) for h in headings if h.text.strip()]
    longest_title = max(title_candidates, key=lambda x: x[1])[0] if title_candidates else "No title found"
    print("Extracted Title:", longest_title)

    paragraphs = soup.find_all('p')
    body_candidates = [p.text.strip() for p in paragraphs if len(p.text.strip()) > 100]
    article_body = ' '.join(body_candidates[:5]) if body_candidates else "No body found"  # Limit to 5 paragraphs
    print("Extracted Body Snippet:", article_body[:200] + "..." if article_body else article_body)

except requests.RequestException as e:
    print("Error fetching URL:", e)

Extracted Title: MB: Ar: SideMenu
Extracted Body Snippet: No body found...
